If you want to save everything: `Click on File -> Save a Copy in Drive`

Now you have it in your drive and can always get back to it!

# Step 1 - Download the dependencies and the PDF files

In [ ]:
# The ! at the beginning is a jupyter notebook specific. It tells the server to execute this as
# shell (instead of python) code.
# pip : package installer for Python

!pip install -q pymupdf
!pip install -q langchain_community
!pip install -q chromadb
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
# Create folders to store the images and the Markdown files
# mkdir: create a new folder

!mkdir image_folder
!mkdir md_docs

In [ ]:
# wget: program to download
# unzip: program to unzip (-o tells to overwrite if files exist)

!wget -q https://github.com/eth-student-project-house/ws-ml-pdf/archive/refs/heads/main.zip
!unzip -q -o /content/main.zip

You can also use your own PDF documents in Drive. Just uncomment and run the next cell so that this file has access to your Drive. We won't have access to your drive.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#Step 2 - Extract the Text and the Images

In [ ]:
pdf_file_folder = "/content/ws-ml-pdf-main/" #Uncomment this line if you use our PDF from Github
#pdf_file_folder = "/content/drive/MyDrive/..."  #Uncomment this line if you use your own PDF from Drive
pdf_name = "olympic_history" #@param {type:"string"}
pdf_file_path = f"{pdf_file_folder}{pdf_name}.pdf"

In [ ]:
import fitz
from langchain_community.document_loaders import PyMuPDFLoader

## Extract the text and save it in .md files

In [ ]:
loader = PyMuPDFLoader(pdf_file_path)
data = loader.load() #extract the text in raw format

for page in data:# iterate over pdf pages
    output_path = f"md_docs/{pdf_name}_page_{page.metadata['page']+1}_{page.metadata['total_pages']}.md" # get the path for the new .md file
    with open(output_path, 'w') as f: # open a new .md file
        f.write(page.page_content) # write in the .md file the raw text
        print(f"Markdown for document {pdf_name}, page {page.metadata['page']+1}/{page.metadata['total_pages']} saved to {output_path}")

## Extract the images as .png files

In [ ]:
doc = fitz.open(pdf_file_path) # load the document

for page_index in range(len(doc)): # iterate over pdf pages
    page = doc[page_index] # get the page

    image_list = page.get_images() # get the images in the page

    # print the number of images found on the page
    if image_list:
        print(f"Found {len(image_list)} images on page {page_index + 1}")
    else:
        print("No images found on page", page_index + 1)

    for image_index, img in enumerate(image_list, start=1): # enumerate the image list
        xref = img[0] # get the XREF of the image
        pix = fitz.Pixmap(doc, xref) # create a Pixmap

        if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
            pix = fitz.Pixmap(fitz.csRGB, pix)
        output_path = f"image_folder/{pdf_name}_page_{page_index+1}-image_{image_index}.png"
        pix.save(output_path) # save the image as png in image_folder
        print(f"Image {image_index} from page {page_index+1} in document {pdf_name} saved to {output_path}")
        pix = None

# Step 3 - Extract the Images information using GPT-4

In [ ]:
prompt = """Describe the following picture as precisely as you can.
            It should contain all the information such that someone can recreate the image from the text explanation.
            Convert tables to markdown tables. Describe charts as best you can.
            If the picture contains people don't describe them, only what they are doing very briefly.
            Don't interpret what you see, only describe, nothing else.
            DO NOT return in a codeblock. Just return the raw text in markdown format."""

In [ ]:
import requests
import base64
from openai import OpenAI

# this openai api key is only valid today. go here to get your own:
# https://platform.openai.com/api-keys
api_key ="sk-proj-jYsmUBNYhrpO34u3Ah7ST3BlbkFJORWMsB6UTJsaySVFL5OI"
openai_client = OpenAI(api_key=api_key)

def image_to_markdown(base64_image):
    # Makes a calll to GPT-4-vision to process an image
    response = openai_client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
              {"type": "text", "text": prompt},
              {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}", "detail": "low"}}
            ],
        }
    ],
    max_tokens =  4096
    )
    return response.choices[0].message.content

def encode_image_to_base64(image_path):
    # Write the text description of the image in a .md file
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

## Process the images and store the result in .md files

In [ ]:
import os
from pathlib import Path

input_path = "image_folder"
output_folder = "md_docs"

images = sorted(Path(input_path).iterdir(), key=lambda x: x.stem) # get all the images in the image_folder
for image_path in images: # Iterate thourgh the images
    output_path = Path(output_folder) / f"{image_path.stem}.md"
    if os.path.exists(output_path): # check if we aleary processed this image
        print(f"File {output_path} already exists")
    else:
        print(f"Processing {image_path.name}...")
        base64_image = encode_image_to_base64(str(image_path)) # encode the image
        markdown_content = image_to_markdown(base64_image) # process the image
        with open(output_path, 'w') as f:
            f.write(markdown_content) #write the description
            print(f"Markdown for {image_path.name} saved to {output_path}")

#Step 4 - Create our DataBase

In [ ]:
import chromadb

from chromadb.config import Settings
from chromadb.utils  import embedding_functions

chroma_client = chromadb.PersistentClient("/content/.chromadb")
# embedding function
default_ef = embedding_functions.DefaultEmbeddingFunction()

collection = chroma_client.get_or_create_collection(name="pdf", embedding_function=default_ef)
print("created new collection")

dataDirectory = "/content/md_docs/"

for filename in os.listdir(dataDirectory): # Iterate through the files in md_docs/
  if filename.endswith('.md'):
    # Open the file and read its contents
    filepath = os.path.join(dataDirectory, filename)
    with open(filepath, 'r') as f:
        fileContent = f.read()

    # Call your function on the file content
    print(f"adding document {filepath}")
    collection.upsert(documents=[fileContent], ids=[filepath])

In [ ]:
collection.peek(limit=1)

# Step 5 - Sample Question + Query the relevant documents

In [ ]:
#### Step 5.1: Question
question = "What were the sport in the ancient olympics?" #@param {type:"string"}
nr_of_results = 6 #@param {type:"slider", min:1, max:10}
#### Step 5.2: Search Vector Database
results = collection.query(query_texts=[question], n_results=nr_of_results)
# line below just outputs
#results


# Step 6 - Ask our Question to the Large Language Model

In [ ]:
# This let's you format the output nicely!
from IPython.display import display, Markdown

## Create our prompt

**Create your Prompt**

- <u>**System:**</u> Sets the behaviour of the assistant, the style of the response
- <u>**User:**</u> Contains our request
- <u>**Assistant:**</u> Can contains response example or history of the previous messages

In [ ]:
### Now tell the language model how to act. This is an example. (it might be a rather dumm instruction given our context!)

# System Prompt Behaviour
behaviour = f"""\
You are a helpful assistant.
"""

# Question + Context
### This is where we add our question ⬇⬇
question = f"""\
CONTEXT: {results['documents'][0]}

QUESTION: {question}
"""

### This is the final prompt
messages=[
    {"role": "system", "content": behaviour},
    {"role": "user", "content": question},
]
## show messages by uncommenting
#messages

In [ ]:
### Call to OPEN AI

# randomness seed (0 no randomness)
temperature = 0.5 # @param {type:"slider", min:0, max:1, step:0.1}
# tokens to expect as response
max_tokens = 1500 # @param {type: "slider", min:100, max: 12000, step: 100}

response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temperature,
    max_tokens=max_tokens,
    top_p=1,
    frequency_penalty=1,
    presence_penalty=1
)

display(Markdown(response.choices[0].message.content))